In [ ]:
import pandas as pd
import numpy as np

# Importing the XGBoost classifier we shall use later.
import xgboost as xgb
from xgboost import XGBClassifier


from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold, cross_validate, RandomizedSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    make_scorer, balanced_accuracy_score
)


from scipy.stats import randint, uniform, norm
from sklearn.metrics import fbeta_score



In [ ]:
# Loading the names for entibiotics, and reading their files.
antibiotics = ['Gentamicin', 'Trimethoprim_Sulfamethoxazole', 'Ciprofloxacin',
                'Ampicillin', 'Cefazolin','Nitrofurantoin','Piperacillin_Tazobactam',
                'Levofloxacin', 'Ceftriaxone']

# Fix a random state for reproducability.
n_randstat = 312 

In [ ]:
# Load and prepare data

for anti in antibiotics:

    all_results = []      #Creating dataframe to store model measurement results later.

    train_df = pd.read_csv(
        f"Final_dataframe-20251015T154934Z-1-001\\Final_dataframe\\{anti}_train_data.csv"
    )

    X = train_df.drop(columns=[anti, 'Year', 'anon_id'])
    y = train_df[anti] - 1  # Convert {1,2} to {0,1}


    # Calculating the positive/negative weight of our target variable set, which shall be used in XGBoost to counter the high imbalance of some antibiotic data.
    neg_count = np.sum(y == 0)
    pos_count = np.sum(y == 1)
    scale_pos_weight_value = neg_count / pos_count 



    # Define base pipeline  
    # Note: for XGBoost, we do not need employ Standard Scaler.
    base_pipeline = Pipeline([
        ('xgb', XGBClassifier(
            objective='binary:logistic',
            eval_metric='auc',
            random_state=n_randstat
        ))
    ])

    
    # Hyperparameter space for random parameter search.
    param_distributions = {
        'xgb__n_estimators': randint(300, 400),      
        'xgb__learning_rate': uniform(0.05, 0.20),   
        'xgb__max_depth': randint(3, 5),
        'xgb__subsample': uniform(0.3, 0.7),
        'xgb__colsample_bytree': uniform(0.3, 0.7),
        'xgb__gamma': uniform(0.1, 1),
        'xgb__scale_pos_weight': [1,0.25*scale_pos_weight_value, 0.5*scale_pos_weight_value, 0.75*scale_pos_weight_value, scale_pos_weight_value]       
    }


    # Scoring setup (all using Fβ)
    scoring = {
        'accuracy': make_scorer(accuracy_score, zero_division=0),
        'recall': make_scorer(recall_score, zero_division=0),         # Maximizing recall is equivalent to minimizing fall negative rate, as they always sum up to be 1.
        'precision': make_scorer(precision_score, zero_division=0),
        'f1': make_scorer(f1_score, average = 'weighted', zero_division=0),      # Using weighted F1 score for model evaluation.
        'balanced_accuracy': make_scorer(balanced_accuracy_score, zero_division = 0),
    
    }



    # Nested CV setup
    outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=n_randstat)    # Outer 5-fold loop for cross-validation.
    inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=n_randstat)    # Inner 3-fold loop for hyper-parameter tuning.

    outer_results = []
    outer_fold = 1

    for train_idx, test_idx in outer_cv.split(X, y):                         # Outer 5-fold loop for cross-validation.
        print(f"\n==============================")
        print(f"🔹 Outer Fold {outer_fold} — training inner search for {anti}")
        print(f"==============================")

        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # ---- Inner hyperparameter tuning ----
        search = RandomizedSearchCV(                      # Defining the randomized searching module.
            estimator=base_pipeline,
            param_distributions=param_distributions,
            n_iter= 10,     # Iteration time for randomized searching.
            scoring= 'f1',  # using weighted f1 for optimization
            cv=inner_cv,    # Using a inner 3-fold for hyper-parameter tuning.
            random_state=n_randstat
        )

        search.fit(X_train, y_train)

        print("Best inner parameters:", search.best_params_)
        print(f"Best inner score:", search.best_score_)



        # ---- Evaluate on outer fold ----
        best_model = search.best_estimator_
        y_pred = best_model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred, zero_division=0)
        precision = precision_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
        fnr = 1 - recall

        outer_results.append({
            'fold': outer_fold,
            'best_params': search.best_params_,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'accuracy': accuracy,
            'FNR': fnr
        })

        outer_fold += 1    
    

    # Aggregate final performance
    outer_df = pd.DataFrame(outer_results)
    print(f"\n\n===== Nested CV Summary for {anti}=====")
    print(outer_df[['fold', 'recall', 'f1', 'FNR']])
    print("Mean Accuracy:", outer_df['accuracy'].mean())
    print("Mean Recall:", outer_df['recall'].mean())                
    print(f"Mean F1:", outer_df['f1'].mean())
    print("Mean false negative rate:", outer_df['FNR'].mean())

    # Save per-antibiotic result CSV
    output_path = f"results_{anti}.csv"
    outer_df.to_csv(output_path, index=False)
    print(f"Saved results to {output_path}")

    # Add to results list
    all_results.append(outer_df)


# Merge all antibiotics' results together
final_df = pd.concat(all_results, ignore_index=True)
final_df.to_csv("all_antibiotics_results.csv", index=False)
print("Saved all results to all_antibiotics_results.csv")


🔹 Outer Fold 1 — training inner search for Gentamicin
Best inner parameters: {'xgb__colsample_bytree': 0.9840134829675653, 'xgb__gamma': 1.0440636313297216, 'xgb__learning_rate': 0.17903579482489707, 'xgb__max_depth': 4, 'xgb__n_estimators': 371, 'xgb__scale_pos_weight': 12.133874239350913, 'xgb__subsample': 0.3821561339972655}
Best inner score: 0.12897892952923815

🔹 Outer Fold 2 — training inner search for Gentamicin
Best inner parameters: {'xgb__colsample_bytree': 0.9840134829675653, 'xgb__gamma': 1.0440636313297216, 'xgb__learning_rate': 0.17903579482489707, 'xgb__max_depth': 4, 'xgb__n_estimators': 371, 'xgb__scale_pos_weight': 12.133874239350913, 'xgb__subsample': 0.3821561339972655}
Best inner score: 0.12962730996904673

🔹 Outer Fold 3 — training inner search for Gentamicin
Best inner parameters: {'xgb__colsample_bytree': 0.9840134829675653, 'xgb__gamma': 1.0440636313297216, 'xgb__learning_rate': 0.17903579482489707, 'xgb__max_depth': 4, 'xgb__n_estimators': 371, 'xgb__scale_po

Exception ignored in: <function Booster.__del__ at 0x0000023CAB819B20>
Traceback (most recent call last):
  File "c:\Anaconda\envs\erdos_ds_environment\Lib\site-packages\xgboost\core.py", line 1797, in __del__
    _check_call(_LIB.XGBoosterFree(self.handle))
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt: 


Best inner parameters: {'xgb__colsample_bytree': 0.9840134829675653, 'xgb__gamma': 1.0440636313297216, 'xgb__learning_rate': 0.17903579482489707, 'xgb__max_depth': 4, 'xgb__n_estimators': 371, 'xgb__scale_pos_weight': 3.375, 'xgb__subsample': 0.3821561339972655}
Best inner score: 0.273759190541028

🔹 Outer Fold 3 — training inner search for Trimethoprim_Sulfamethoxazole
Best inner parameters: {'xgb__colsample_bytree': 0.9840134829675653, 'xgb__gamma': 1.0440636313297216, 'xgb__learning_rate': 0.17903579482489707, 'xgb__max_depth': 4, 'xgb__n_estimators': 371, 'xgb__scale_pos_weight': 3.375, 'xgb__subsample': 0.3821561339972655}
Best inner score: 0.3035168769511159

🔹 Outer Fold 4 — training inner search for Trimethoprim_Sulfamethoxazole
Best inner parameters: {'xgb__colsample_bytree': 0.9840134829675653, 'xgb__gamma': 1.0440636313297216, 'xgb__learning_rate': 0.17903579482489707, 'xgb__max_depth': 4, 'xgb__n_estimators': 371, 'xgb__scale_pos_weight': 3.375, 'xgb__subsample': 0.38215613